 Part a,b,c - Written in the writeup attached.

Part d)

In [3]:
import numpy as np

# Load data
ratings_matrix = np.loadtxt("user-shows.txt")  # Path to user-show matrix
with open("shows.txt", "r") as f:
    tv_show_titles = [line.strip() for line in f.readlines()]

# Mask the first 100 shows for Alex (user at index 499)
alex_user_index = 499
masked_ratings = ratings_matrix.copy()
masked_ratings[alex_user_index, :100] = 0  # Hide first 100 shows for prediction

# Compute P (user degree) and Q (item degree) matrices
user_likes_count = np.sum(masked_ratings, axis=1)
item_popularity = np.sum(masked_ratings, axis=0)
P = np.diag(user_likes_count)
Q = np.diag(item_popularity)

# Compute P^(-1/2) and Q^(-1/2)
P_inv_sqrt = np.diag([1 / np.sqrt(val) if val != 0 else 0 for val in user_likes_count])
Q_inv_sqrt = np.diag([1 / np.sqrt(val) if val != 0 else 0 for val in item_popularity])

# ---------- User-User Collaborative Filtering ----------
user_similarity_matrix = P_inv_sqrt @ masked_ratings @ masked_ratings.T @ P_inv_sqrt
user_cf_scores = user_similarity_matrix @ masked_ratings
alex_user_cf_scores = user_cf_scores[alex_user_index, :100]

# Get top 10 recommendations from first 100 shows
top10_user_cf_indices = np.argsort(-alex_user_cf_scores)[:10]
top10_user_cf_titles = [tv_show_titles[i] for i in top10_user_cf_indices]

# ---------- Item-Item Collaborative Filtering ----------
item_similarity_matrix = Q_inv_sqrt @ masked_ratings.T @ masked_ratings @ Q_inv_sqrt
item_cf_scores = masked_ratings @ item_similarity_matrix
alex_item_cf_scores = item_cf_scores[alex_user_index, :100]

# Get top 10 recommendations from first 100 shows
top10_item_cf_indices = np.argsort(-alex_item_cf_scores)[:10]
top10_item_cf_titles = [tv_show_titles[i] for i in top10_item_cf_indices]

# ---------- Print Recommendations ----------
print("Top 10 User-User CF Recommendations for Alex (from first 100 shows):")
for i, title in enumerate(top10_user_cf_titles, 1):
    print(f"{i}. {title}")

print("\nTop 10 Item-Item CF Recommendations for Alex (from first 100 shows):")
for i, title in enumerate(top10_item_cf_titles, 1):
    print(f"{i}. {title}")

# ---------- Sanity Check ----------
max_user_cf_score = np.max(alex_user_cf_scores)
max_item_cf_score = np.max(alex_item_cf_scores)

print(f"\nSanity Check:")
print(f"Max User-User score (should be > 900): {max_user_cf_score}")
print(f"Max Item-Item score (should be > 31): {max_item_cf_score}")


Top 10 User-User CF Recommendations for Alex (from first 100 shows):
1. "2009 NCAA Basketball Tournament"
2. "Family Guy"
3. "FOX 28 News at 10pm"
4. "NBC 4 at Eleven"
5. "Friends"
6. "Succession"
7. "Two and a Half Men"
8. "Everybody Loves Raymond"
9. "Law & Order: Special Victims Unit"
10. "NBC 4 Today "

Top 10 Item-Item CF Recommendations for Alex (from first 100 shows):
1. "2009 NCAA Basketball Tournament"
2. "Family Guy"
3. "NBC 4 at Eleven"
4. "FOX 28 News at 10pm"
5. "Two and a Half Men"
6. "Friends"
7. "Succession"
8. "Everybody Loves Raymond"
9. "Law & Order: Special Victims Unit"
10. "CBS Evening News"

Sanity Check:
Max User-User score (should be > 900): 908.4800534761279
Max Item-Item score (should be > 31): 31.364701678342406
